In [ ]:
%pip install pandas
%pip install numpy 
%pip install matplotlib
%pip install seaborn

In [11]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import re as re
from collections import Counter

In [8]:
# Carregar cada CSV limpo
df_netflix = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_netflix_clean.csv')
df_hbo = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_hbo_clean.csv')
df_apple = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/df_apple_clean.csv')
df_prime = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_prime_video_clean.csv')



print("📂 DATASETS CARREGADOS:")
print(f"Netflix: {len(df_netflix):,} registros")
print(f"HBO Max: {len(df_hbo):,} registros") 
print(f"Apple TV: {len(df_apple):,} registros")
print(f"Prime Video: {len(df_prime):,} registros")

📂 DATASETS CARREGADOS:
Netflix: 21,082 registros
HBO Max: 9,575 registros
Apple TV: 18,131 registros
Prime Video: 71,280 registros


In [9]:
# Carregar os dados limpos 
df_netflix = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_netflix_clean.csv')
df_hbo = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_hbo_clean.csv')
df_apple = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/df_apple_clean.csv')
df_prime = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_prime_video_clean.csv')

# Adicionar coluna de plataforma
df_netflix['platform'] = 'Netflix'
df_hbo['platform'] = 'HBO Max'
df_apple['platform'] = 'Apple TV+'
df_prime['platform'] = 'Prime Video'

In [10]:

# ==========================================
# UNINDO OS DATASETS
# ==========================================

platforms = {
    'Netflix': df_netflix,
    'HBO Max': df_hbo, 
    'Apple TV': df_apple,
    'Prime Video': df_prime
}
# Concatenar todos os DataFrames em um único DataFrame
df_streaming = pd.concat([df_netflix, df_hbo, df_apple, df_prime], ignore_index=True)


print(f"\n🎬 DATASET UNIFICADO CRIADO:")
print(f"Total de registros: {len(df_streaming):,}")
print(f"Total de colunas: {len(df_streaming.columns)}")
print(f"Colunas: {list(df_streaming.columns)}")



🎬 DATASET UNIFICADO CRIADO:
Total de registros: 120,068
Total de colunas: 8
Colunas: ['title', 'type', 'genres', 'releaseYear', 'imdbId', 'imdbAverageRating', 'imdbNumVotes', 'platform']


In [12]:
def extract_individual_genres(df, platform_name):
    """Extrai gêneros individuais e conta por plataforma"""
    all_genres = []
    
    for genres_str in df['genres'].dropna():
        if pd.notna(genres_str):
            genres_list = re.split(r'[,;|]', str(genres_str))
            clean_genres = [genre.strip() for genre in genres_list if genre.strip()]
            all_genres.extend(clean_genres)
    
    genre_counts = Counter(all_genres)
    total_occurrences = sum(genre_counts.values())
    
    # Converter para DataFrame com percentuais
    genre_df = pd.DataFrame(list(genre_counts.items()), columns=['genre', 'count'])
    genre_df['percentage'] = (genre_df['count'] / total_occurrences) * 100
    genre_df['platform'] = platform_name
    genre_df = genre_df.sort_values('percentage', ascending=False)
    
    return genre_df

In [13]:
# Extrair dados de gênero por plataforma
netflix_genres = extract_individual_genres(df_netflix, 'Netflix')
hbo_genres = extract_individual_genres(df_hbo, 'HBO Max')
apple_genres = extract_individual_genres(df_apple, 'Apple TV+')
prime_genres = extract_individual_genres(df_prime, 'Prime Video')

# Combinar dados de gêneros
all_genres_data = pd.concat([netflix_genres, hbo_genres, apple_genres, prime_genres], ignore_index=True)